안녕하세요?<br>
저희는 LG Aimers 4기 아주 수학 신세계 트리오(김동건, 김지연, 명서준)팀 입니다.

코드 순서는 크게
1. 전처리
2. 모델<br>

순으로 이루어집니다.<br>

전처리는 각 column별 전처리를 함수화 하고 나중에 모든 column에 대한 전처리를 하나의 함수로 모아 실행하였습니다.

In [1]:
# 엘리스 주피터 환경과 마찬가지로 넘파이와 판다스, 사이킷런은 설치되어있다고 가정하고 시작하겠습니다.
# !pip install catboost
# !pip install xgboost
# !pip install imblearn

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

import xgboost as xgb
import catboost as cat
from sklearn.ensemble import VotingClassifier
from imblearn.over_sampling import ADASYN

import warnings
warnings.filterwarnings('ignore')

이제부터 본격적으로 전처리 코드에 들어가도록 하겠습니다.

In [3]:
# 저희의 EDA 결과, customer_country는 customer_country.1 칼럼과 거의 일치하여 drop을 결정하였고
# product_subcategory와 business_subarea는 결측치가 매우 많아 drop을 결정하였습니다.

def pp_drop(df) :
    df.drop(['customer_country','product_subcategory','business_subarea'], axis = 1, inplace = True)
    print('drop이 완료되었습니다.')

In [4]:
# train data 약 6만개중에 business_unit에 CM이 2개밖에 되지 않아 주요 business_unit인 ID, IT, AS 외에는 Others로 넣었습니다.

def pp2(df) :
    df.loc[(df['business_unit']!='ID')&(df['business_unit']!='AS')&(df['business_unit']!='IT'), 'business_unit'] = 'Others'

In [5]:
# pp6은 customer_type에 대한 전처리 입니다.
# 우선 같은 의미이지만 띄워쓰기나, 대소문자로 구별되는 것들을 하나로 묶은 다음
# .value_counts()후 1개짜리 데이터는 'Others'로 빼고 NaN값은 'none'값으로 처리하였습니다.

def pp6(df) :
    df.loc[df['customer_type']=='End-Customer', 'customer_type'] = 'End Customer'

    df.loc[(df['customer_type']=='Specifier / Influencer')|(df['customer_type']=='Specifier/ Influencer'), 'customer_type'] = 'Specifier/Influencer'

    df.loc[df['customer_type']=='Installer/Contractor', 'customer_type'] = 'Installer'

    df.loc[df['customer_type']=='Home Owner', 'customer_type'] = 'Homeowner'

    df.loc[df['customer_type']=='Consultant', 'customer_type'] = 'Architect/Consultant'

    df.loc[df['customer_type']=='End-user', 'customer_type'] = 'End user'

    df.loc[df['customer_type']=='Distributor', 'customer_type'] = 'Dealer/Distributor'

    df.loc[df['customer_type']=='Commercial end-user', 'customer_type'] = 'End user'
    
    df.loc[df['customer_type']=='End user', 'customer_type'] = 'End Customer'

    df.loc[df['customer_type']=='Other', 'customer_type'] = 'Others'
    df.loc[df['customer_type']=='Etc.', 'customer_type'] = 'Others'
    df.loc[df['customer_type']=='System Integrator', 'customer_type'] = 'Others'
    df.loc[df['customer_type']=='Administrator', 'customer_type'] = 'Others'
    df.loc[df['customer_type']=='Technical Assistant', 'customer_type'] = 'Technician'
    df.loc[df['customer_type']=='Software / Solution Provider', 'customer_type'] = 'Software/Solution Provider'
    df.loc[df['customer_type'] != df['customer_type'], 'customer_type'] = 'none'
    
    if df['customer_type'].isna().sum() == 0 :
        print('customer_type 전처리가 완료되었습니다.(채워진 none값 주의)')

In [6]:
# pp6_hu는 customer_type의 추가 전처리 부분입니다.
# pp6을 실행한 뒤에 .value_counts()가 약 50개 미만인 값들에 대해서
# LG 홈페이지의 Inquiry To Buy의 양식을 참고하여 Customer Subtype의 값들이라 생각되는 것들을 상위 Customer Type으로 옮겨주고 판단하기 어려운 값들은 Others로 옮겨주었습니다.

def pp6_hu(df) :
    df.loc[df['customer_type']=='Developer', 'customer_type'] = 'Solution Eco-Partner'
    df.loc[df['customer_type']=='Homeowner', 'customer_type'] = 'Others'
    df.loc[df['customer_type']=='Engineer', 'customer_type'] = 'Others'
    df.loc[df['customer_type']=='HVAC Engineer', 'customer_type'] = 'Others'
    df.loc[df['customer_type']=='Dealer/Distributor', 'customer_type'] = 'Channel Partner'
    df.loc[df['customer_type']=='Reseller', 'customer_type'] = 'Channel Partner'
    df.loc[df['customer_type']=='Manager / Director', 'customer_type'] = 'Specifier/Influencer'
    df.loc[df['customer_type']=='Architect/Consultant', 'customer_type'] = 'Specifier/Influencer'
    df.loc[df['customer_type']=='Installer', 'customer_type'] = 'Specifier/Influencer'
    df.loc[df['customer_type']=='Interior Designer', 'customer_type'] = 'Specifier/Influencer'
    df.loc[df['customer_type']=='Technician', 'customer_type'] = 'Specifier/Influencer'
    df.loc[df['customer_type']=='Software/Solution Provider', 'customer_type'] = 'Solution Eco-Partner'
    df.loc[df['customer_type']=='Corporate', 'customer_type'] = 'Others'
    
    df.loc[(df['customer_position']!='CEO/Founder')&(df['customer_position']!='C-level Executive')&(df['customer_position']!='Manager')&(df['customer_position']!='Director')&(df['customer_position']!='Associate/Analyst')&(df['customer_position']!='Partner')&(df['customer_position']!='Entry Level')&(df['customer_position']!='Trainee')&(df['customer_position']!='Vice President')&(df['customer_position']!='Intern')&(df['customer_position']!='none')&(df['customer_position']!='others'), 'customer_position'] = 'none'
    
    print('customer_type, customer_pos 후처리가 완료되었습니다.')

In [7]:
# pp8은 historical_existing_cnt에 대한 전처리 입니다.
# historical_existing_cnt는 이전 영업전환횟수로 결측치는 이전 영업 전환 횟수를 측정할 수 없는 신규 고객, 그외 값은 기존 고객이라고 생각하였습니다.
# 때문에 결측치를 -1로 채운 뒤, -1의 값을 갖는 친구들에 대해서는 new를, 0의 값을 갖는 친구들에 대해서는 영업전환이 없는 zero라는 값을 부여하였고
# 그 뒤에는 kde plot과 특정 범위(구간) 별 True 비율을 확인해가면서 이전 영업전환 횟수가 적다(little), 중간(middel), 많다(many), 아주많다(so many)로 나누었습니다.

def pp8(df):
    df.loc[df['historical_existing_cnt'] != df['historical_existing_cnt'], 'historical_existing_cnt'] = -1
    
    idx0 = df.loc[df["historical_existing_cnt"] == -1, "historical_existing_cnt"].index
    idx1 = df.loc[df["historical_existing_cnt"] == 0, "historical_existing_cnt"].index
    idx2 = df.loc[(df["historical_existing_cnt"] > 0) & (df["historical_existing_cnt"] < 30), "historical_existing_cnt"].index
    idx3 = df.loc[(df["historical_existing_cnt"] >= 30) & (df["historical_existing_cnt"] < 80), "historical_existing_cnt"].index
    idx4 = df.loc[(df["historical_existing_cnt"] >= 80) & (df["historical_existing_cnt"] < 130), "historical_existing_cnt"].index
    idx5 = df.loc[(df["historical_existing_cnt"] >= 130), "historical_existing_cnt"].index
    
    df.loc[idx0, "historical_existing_cnt"] = "new"
    df.loc[idx1, "historical_existing_cnt"] = "zero"
    df.loc[idx2, "historical_existing_cnt"] = "little"
    df.loc[idx3, "historical_existing_cnt"] = "middle"
    df.loc[idx4, "historical_existing_cnt"] = "many"
    df.loc[idx5, "historical_existing_cnt"] = "so many"
    
    if df['historical_existing_cnt'].isna().sum() == 0 :
        print('historical_existing_cnt 전처리가 완료되었습니다.')

In [8]:
# pp12는 customer_job에 대한 전처리 함수입니다.
# customer_job 또한 같은 의미 의지만 대소문자, 띄워쓰기, 다른나라 언어로 구별되는 같은 값으로 묶어준 다음
# LG 홈페이지의 Inquiry To Buy를 참고하여 전처리 후에도 의미가 비슷해 보이는 값들은 같은 값으로 넣어주었으며 
# .value_counts()가 적은 친구나, 중요해보이지 않은 값들에 대해서는 Others로 넣어주었습니다.

def pp12(df) :
    df.loc[df['customer_job'] != df['customer_job'], 'customer_job'] = 'none'
    df.loc[df['customer_job']=='other', 'customer_job'] = 'others'
    df.loc[df['customer_job']=='engineer', 'customer_job'] = 'engineering'
    df.loc[df['customer_job']=='engineering & technical', 'customer_job'] = 'engineering'
    df.loc[df['customer_job']=='decision maker', 'customer_job'] = 'executive'
    df.loc[df['customer_job']=='program_and_project_management', 'customer_job'] = 'program and project management'
    df.loc[df['customer_job']=='it - information technology', 'customer_job'] = 'information technology'
    df.loc[df['customer_job']=='it', 'customer_job'] = 'information technology'
    df.loc[df['customer_job']=='buyer', 'customer_job'] = 'purchasing'
    df.loc[df['customer_job']=='értékesítés', 'customer_job'] = 'sales'
    df.loc[df['customer_job']=='coo', 'customer_job'] = 'executive'
    df.loc[df['customer_job']=='pathologist', 'customer_job'] = 'medical'
    
    for i, job in enumerate(df['customer_job']):
        if 'engine' in job :
            df['customer_job'][i] = 'engineering'
        elif 'consult' in job :
            df['customer_job'][i] = 'consulting'
        elif ('art' in job) or ('design' in job) :
            df['customer_job'][i] = 'arts and design'
        elif 'director' in job :
            df['customer_job'][i] = 'director'
        elif 'marketing' in job :
            df['customer_job'][i] = 'marketing'
        elif 'purcha' in job :
            df['customer_job'][i] = 'purchasing'
        elif ('teach' in job) or ('edu' in job) :
            df['customer_job'][i] = 'education'
        elif ('ceo' in job) or ('own' in job) :
            df['customer_job'][i] = 'ceo'
        elif ('health' in job) or ('clinic' in job) or ('medi' in job) or ('surger' in job)or ('hospi' in job) :
            df['customer_job'][i] = 'medical'
        elif 'account' in job :
            df['customer_job'][i] = 'accounting'
            
    df_pc = df['customer_job'].value_counts()
    desired_values = df_pc[df['customer_job'].value_counts() < 20]
    idx_list = desired_values.index
    
    for i, job in enumerate(df['customer_job']) :
        if job in idx_list :
            if 'manage' in job :
                df['customer_job'][i] = 'manager'
            elif 'tech' in job :
                df['customer_job'][i] = 'technical'
            elif 'lead' in job :
                df['customer_job'][i] = 'executive'
            elif 'head' in job :
                df['customer_job'][i] = 'executive'
            elif 'chief' in job :
                df['customer_job'][i] = 'executive'
    
    counts = df['customer_job'].value_counts()
    mask = df['customer_job'].isin(counts[counts < 20].index)
    df['customer_job'] = df['customer_job'].where(~mask, 'others')
    
    df.loc[df['customer_job']=='gm', 'customer_job'] = 'others'
    df.loc[df['customer_job']=='finanzen', 'customer_job'] = 'finance'
    df.loc[df['customer_job']=='general manager', 'customer_job'] = 'manager'
    df.loc[df['customer_job']=='general management', 'customer_job'] = 'manager'
    df.loc[df['customer_job']=='vertrieb', 'customer_job'] = 'sales'
    df.loc[df['customer_job']=='programm-_und_projektmanagement', 'customer_job'] = 'program and project management'
    df.loc[df['customer_job']=='lead', 'customer_job'] = 'executive'
    df.loc[df['customer_job']=='sonstiges', 'customer_job'] = 'others'
    df.loc[df['customer_job']=='quality assurance', 'customer_job'] = 'others'
    df.loc[df['customer_job']=='curation', 'customer_job'] = 'others'
    df.loc[df['customer_job']=='technical', 'customer_job'] = 'others'
    
    if df['customer_job'].isna().sum() == 0 :
        print('customer_job 전처리가 완료되었습니다.(채워진 none값 주의)')

In [9]:
# pp13은 lead_desc_length에 대한 전처리입니다.
# 값이 2이하인 친구들에 대해서는 의미가 없거나, 따로 요청사항이 없는 것이라 판단 하여 'No'라는 값을 주었고
# 그 뒤에 kde plot과 특정 범위들의 True 비율들을 확인해가면서 요청사항이 짧다(Short), 길다(Long)으로 나누었습니다.

def pp13(df) :
    idx1 = df.loc[(df['lead_desc_length'] <= 2), 'lead_desc_length'].index
    idx2 = df.loc[(df['lead_desc_length'] > 2) & (df['lead_desc_length'] <= 130), 'lead_desc_length'].index
    idx3 = df.loc[(df['lead_desc_length'] > 130), 'lead_desc_length'].index
    
    df.loc[idx1, "lead_desc_length"] = "No"
    df.loc[idx2, "lead_desc_length"] = "Short"
    df.loc[idx3, "lead_desc_length"] = "Long"
    
    if df['lead_desc_length'].isna().sum() == 0 :
        print('lead_desc_length 전처리가 완료되었습니다.')

In [10]:
# pp14는 inquiry_type의 전처리 함수입니다.

# 다른 칼럼과 마찬가지로 띄워쓰기, 대소문자로 구별되지만 의미가 같은 값들을 같아지게끔 전처리를 한 후에
# NaN값은 'none' 값으로 채운 후
# .value_counts()가 매우 적은 친구들에게는 Others의 값을 부여하였습니다.
# inquiry_type도 LG 홈페이지의 Inquiry To Buy를 참고하여 해당 양식에 없는 값은 지울까 고민을 많이 했지만, 
# 마지막엔 그들의 .value_counts()의 수도 어느정도 있고, 해당 inquiry_type의 값이 의미하는 바가 MQL 데이터 쪽에서 중요해보여 살려두었습니다.(해당 부분은 주석처리 하였습니다.)

def pp14(df) :
    df.loc[(df['inquiry_type']=='Quotation or purchase consultation')|
           (df['inquiry_type']=='quotation_')|
           (df['inquiry_type']=='Quotation or Purchase consultation')|
           (df['inquiry_type']=='Quotation or Purchase Consultation')|
           (df['inquiry_type']=='Request for quotation or purchase')|
           (df['inquiry_type']=='quotation_or_purchase_consultation')|
           (df['inquiry_type']=='Purchase')|
           (df['inquiry_type']=='Purchase or Quotation'), 'inquiry_type'] = 'Quotation or Purchase Consultation'
    
    df.loc[(df['inquiry_type']=='Sales Inquiry')|
           (df['inquiry_type']=='sales')|
           (df['inquiry_type']=='Sales inquiry'), 'inquiry_type'] = 'Sales Inquiry'
    
    df.loc[(df['inquiry_type']=='Technical Consultation')|
           (df['inquiry_type']=='Usage or technical consultation')|
           (df['inquiry_type']=='Usage or Technical Consultation')|
           (df['inquiry_type']=='usage or technical consultation')|
           (df['inquiry_type']=='Technical Support')|
           (df['inquiry_type']=='Request for technical consulting')|
           (df['inquiry_type']=='usage_or_technical_consultation')|
           (df['inquiry_type']=='technical')|
           (df['inquiry_type']=='technical_consultaiton'), 'inquiry_type'] = 'Usage or Technical Consultation'
    
    df.loc[(df['inquiry_type']!='Quotation or Purchase Consultation')&
           (df['inquiry_type']!='Sales Inquiry')&
           (df['inquiry_type']!='Usage or Technical Consultation')&
           (df['inquiry_type']!='Request for Partnership')&
           (df['inquiry_type']!='Product Information')&
           (df['inquiry_type']!='Services')&
           (df['inquiry_type']!='Request a Demo')&
           (df['inquiry_type']!='Request for Distributorship')&
           (df['inquiry_type']!='Trainings')&
           (df['inquiry_type']!='Customer Suggestions'), 'inquiry_type'] = 'Others'
    
    df.loc[df['inquiry_type'] != df['inquiry_type'], 'inquiry_type'] = 'none'
    
    # df.loc[df['inquiry_type']=='Request for Distributorship', 'inquiry_type'] = 'Others'
    # df.loc[df['inquiry_type']=='Customer Suggestions', 'inquiry_type'] = 'Others'
    # df.loc[df['inquiry_type']=='Request a Demo', 'inquiry_type'] = 'Others'
    # df.loc[df['inquiry_type']=='Trainings', 'inquiry_type'] = 'Others'
    # df.loc[df['inquiry_type']=='Services', 'inquiry_type'] = 'Others'
    
    if df['inquiry_type'].isna().sum() == 0 :
        print('inquiry_type 전처리가 완료되었습니다.(채워진 none값 주의)')

In [11]:
# pp15는 product_category의 전처리 함수입니다.

# 다른 칼럼과 마찬가지로 띄워쓰기, 대소문자로 구별되지만 의미가 같은 값들을 같아지게끔 전처리를 한 후에
# 중국어, 헝가리어, 베트남어, 힌두어 등등 정말 많은 값들에 대해서는 하나하나 번역기를 돌려가며 그에 맞는 값을 주었습니다.
# 그 후에 모델명이 적여힜는 친구들에 대해서도 LG 홈페이지에서 해당 모델을 일일이 찾아가며 최대한 그에 해당하는 카테고리 값을 주었습니다.
# 그 후에 .value_counts()의 값이 낮은 친구들에 대해서는 몇몇 키워드를 정해 그에 맞는 카테고리 값을 넣어주었으며
# 그 외에는 'others'의 값을, 결측치에는 'none' 값을 넣어주었습니다. 

def pp15(df) :
    df.loc[df['product_category']=='etc.', 'product_category'] = 'others'
    df.loc[df['product_category']=='other', 'product_category'] = 'others'
    df.loc[df['product_category']=='ฯลฯ', 'product_category'] = 'others'
    df.loc[df['product_category']=='khác', 'product_category'] = 'others'
    df.loc[df['product_category']=='outros', 'product_category'] = 'others'

    df.loc[df['product_category']=='lg one:quick series', 'product_category'] = 'one quick series'
    df.loc[df['product_category']=='one:quick series', 'product_category'] = 'one quick series'
    df.loc[df['product_category']=='onequick series', 'product_category'] = 'one quick series'
    df.loc[df['product_category']=='lg one:quick', 'product_category'] = 'one quick series'
    df.loc[df['product_category']=='lg one:quick', 'product_category'] = 'one quick series'
    df.loc[df['product_category']=='one quick:flex', 'product_category'] = 'one quick series'

    df.loc[df['product_category']=='all lg vrf systems', 'product_category'] = 'vrf'
    df.loc[df['product_category']=='systèmes de débit à réfrigérant variable (drv)', 'product_category'] = 'vrf'
    df.loc[df['product_category']=='điều hòa trung tâm vrf', 'product_category'] = 'vrf'
    df.loc[df['product_category']=='نظام التدفق المتغيرvrf', 'product_category'] = 'vrf'
    df.loc[df['product_category']=='điều hòa trung tâm multi', 'product_category'] = 'vrf'

    df.loc[df['product_category']=='a thermodynamic water heater', 'product_category'] = 'heating'
    df.loc[df['product_category']=='חימום', 'product_category'] = 'heating'
    df.loc[df['product_category']=='حلول التدفئة', 'product_category'] = 'heating'
    df.loc[df['product_category']=='ogrzewanie (pompy ciepła)', 'product_category'] = 'heating'
    df.loc[df['product_category']=='isıtma', 'product_category'] = 'heating'
    df.loc[df['product_category']=='calefacción', 'product_category'] = 'heating'

    df.loc[df['product_category']=='aire acondicionado residencial', 'product_category'] = 'air care'
    df.loc[df['product_category']=='تكييف وتبريد', 'product_category'] = 'air care'
    df.loc[df['product_category']=='מזגנים למקום מגורים', 'product_category'] = 'air care'
    df.loc[df['product_category']=='تكييفات', 'product_category'] = 'air care'
    df.loc[df['product_category']=='เครื่องปรับอากาศเผื่อที่อยู่อาศัย', 'product_category'] = 'air care'
    df.loc[df['product_category']=='điều hòa cục bộ', 'product_category'] = 'air care'
    df.loc[df['product_category']=='điều hòa gia dụng', 'product_category'] = 'air care'
    df.loc[df['product_category']=='مبرد (تشيلر)', 'product_category'] = 'air care'
    df.loc[df['product_category']=='chiller', 'product_category'] = 'air care'
    df.loc[df['product_category']=='điều hòa trung tâm chiller', 'product_category'] = 'air care'
    df.loc[df['product_category']=='ar condicionado residencial', 'product_category'] = 'air care'
    df.loc[df['product_category']=='teto ou cassete inverter', 'product_category'] = 'air care'
    df.loc[df['product_category']=='residential air conditioner', 'product_category'] = 'air care'
    df.loc[df['product_category']=='soğutucu ', 'product_category'] = 'air care'
    df.loc[df['product_category']=='climatiseur résidentiel ', 'product_category'] = 'air care'

    df.loc[df['product_category']=='multi v 5 air', 'product_category'] = 'multi-split'
    df.loc[df['product_category']=='multi v water 5', 'product_category'] = 'multi-split'
    df.loc[df['product_category']=='פיצול מרובה', 'product_category'] = 'multi-split' 

    df.loc[df['product_category']=='high brightness', 'product_category'] = 'high brightness signage'
    df.loc[df['product_category']=='video wall', 'product_category'] = 'video wall signage'
    df.loc[df['product_category']=='55vm5j-h', 'product_category'] = 'video wall signage'
    df.loc[df['product_category']=='49vl5g-m', 'product_category'] = 'video wall signage'
    df.loc[df['product_category']=='55vm5e-a', 'product_category'] = 'video wall signage'
    df.loc[df['product_category']=='55svh7f-a', 'product_category'] = 'video wall signage'
    df.loc[df['product_category']=='49vl5f', 'product_category'] = 'video wall signage'
    df.loc[df['product_category']=='49vl5g-m.awzm', 'product_category'] = 'video wall signage'
    
    df.loc[df['product_category']=='laec15', 'product_category'] = 'led signage'
    df.loc[df['product_category']=='gsca046', 'product_category'] = 'led signage'
    df.loc[df['product_category']=='ledallinone', 'product_category'] = 'led signage'
    df.loc[df['product_category']=='oled 顯示屏', 'product_category'] = 'oled signage'
    df.loc[df['product_category']=='標準顯示屏', 'product_category'] = 'standard signage'
    df.loc[df['product_category']=='led 顯示屏', 'product_category'] = 'led signage'
    df.loc[df['product_category']=='高亮度顯示屏', 'product_category'] = 'high brightness signage'
    df.loc[df['product_category']=='互動式顯示屏', 'product_category'] = 'interactive signage'
    df.loc[df['product_category']=='特別顯示屏', 'product_category'] = 'special signage'

    df.loc[df['product_category']=='酒店電視', 'product_category'] = 'hotel tv'
    df.loc[df['product_category']=='醫院電視', 'product_category'] = 'hotel tv'

    df.loc[df['product_category']=='ur640s', 'product_category'] = 'standard signage'
    df.loc[df['product_category']=='ur640', 'product_category'] = 'standard signage'
    df.loc[df['product_category']=='smart tv signage', 'product_category'] = 'standard signage'
    df.loc[df['product_category']=='32lq621cbsb.awz', 'product_category'] = 'commercial tv'
    df.loc[df['product_category']=='55uq801c0sb.bwz', 'product_category'] = 'hotel tv'
    df.loc[df['product_category']=='50uq801c0sb.bwz', 'product_category'] = 'hotel tv'
    df.loc[df['product_category']=='43uq751c0sb.bwz', 'product_category'] = 'hotel tv'
    df.loc[df['product_category']=='43uq751c0sf.bwz', 'product_category'] = 'hotel tv'
    df.loc[df['product_category']=='55us660h0sd.bwz', 'product_category'] = 'hotel tv'
    df.loc[df['product_category']=='43us660h0sd.awz', 'product_category'] = 'hotel tv'
    df.loc[df['product_category']=='43us660h (na)', 'product_category'] = 'hotel tv'
    df.loc[df['product_category']=='50us660h0sd.bwz', 'product_category'] = 'hotel tv'
    df.loc[df['product_category']=='hoteleria_us670h', 'product_category'] = 'hotel tv'

    df.loc[df['product_category']=='28mq780', 'product_category'] = 'monitor'
    df.loc[df['product_category']=='98uh5e', 'product_category'] = 'standard signage'
    df.loc[df['product_category']=='86uh5f', 'product_category'] = 'standard signage'
    df.loc[df['product_category']=='49uh / 49x', 'product_category'] = 'standard signage'
    df.loc[df['product_category']=='49uh / 49xf', 'product_category'] = 'standard signage'

    df.loc[df['product_category']=='55tc3d', 'product_category'] = 'interactive digital board'

    df.loc[df['product_category']=='software solution', 'product_category'] = 'software'
    df.loc[df['product_category']=='軟體', 'product_category'] = 'software'
    df.loc[df['product_category']=='medical displays', 'product_category'] = 'medical display'
    
    df['product_category'] = df['product_category'].fillna('none')

    for i, ctr in enumerate(df['product_category']) :
        if 'vrf' in ctr :
            df['product_category'][i] = 'vrf'
        elif 'rac' in ctr :
            df['product_category'][i] = 'rac'
        
    df_pc = df['product_category'].value_counts()
    desired_values = df_pc[df['product_category'].value_counts() < 40]
    idx_list = desired_values.index

    for i, ctr in enumerate(df['product_category']) :
        if ctr in idx_list :
            if 'quick' in ctr :
                df['product_category'][i] = 'one quick series'
            elif ('air' in ctr) or ('chiller' in ctr) :
                df['product_category'][i] = 'air care'
            elif ('vedio' in ctr) or ('wall' in ctr) :
                df['product_category'][i] = 'video wall signage'
            elif 'monitor' in ctr :
                df['product_category'][i] = 'monitor'
            elif 'multi' in ctr :
                df['product_category'][i] = 'multi-split'
            elif 'projector' in ctr :
                df['product_category'][i] = 'projector'
            elif 'software' in ctr :
                df['product_category'][i] = 'software'
            elif 'heat' in ctr :
                df['product_category'][i] = 'heating'
            elif 'tv' in ctr :
                df['product_category'][i] = 'commercial tv'
            elif 'display' in ctr :
                df['product_category'][i] = 'commercial display'
            else :
                df['product_category'][i] = 'others'
                
    df.loc[df['product_category']=='technical support', 'product_category'] = 'others'
    df.loc[df['product_category']=='pc', 'product_category'] = 'others'
    df.loc[df['product_category']=='multi inverter', 'product_category'] = 'others'
                
    if df['product_category'].isna().sum() == 0 :
        print('product_category의 전처리가 완료되었습니다.(채워진 none값 주의)')

In [12]:
# pp15_hu는 pp15를 하기 전에 수행하는 함수입니다.
# 저희는 나중에 pp17에서 product_modelname을 쓴 사람들에게 가중치를 주는 전처리를 하게 됩니다.
# 하지만 EDA 과정속에서 product_category에 모델명을 적어놓고서 product_modelname에 모델명이 적혀있지 않은 고객이 있다는 것을 발견했고
# 때문에 product_category에 모델명을 써놓은 사람에게도 나중에 product_modelname의 가중치에 추가되게끔 해주었습니다.

def pp15_hu(df) :
    df.loc[df['product_category']=='55vm5j-h', 'product_modelname'] = 1
    df.loc[df['product_category']=='49vl5g-m', 'product_modelname'] = 1
    df.loc[df['product_category']=='55vm5e-a', 'product_modelname'] = 1
    df.loc[df['product_category']=='55svh7f-a', 'product_modelname'] = 1
    df.loc[df['product_category']=='49vl5f', 'product_modelname'] = 1
    df.loc[df['product_category']=='49vl5g-m.awzm', 'product_modelname'] = 1

    df.loc[df['product_category']=='ur640s', 'product_modelname'] = 1
    df.loc[df['product_category']=='ur640', 'product_modelname'] = 1
    df.loc[df['product_category']=='32lq621cbsb.awz', 'product_modelname'] = 1
    df.loc[df['product_category']=='55uq801c0sb.bwz', 'product_modelname'] = 1
    df.loc[df['product_category']=='50uq801c0sb.bwz', 'product_modelname'] = 1
    df.loc[df['product_category']=='43uq751c0sb.bwz', 'product_modelname'] = 1
    df.loc[df['product_category']=='43uq751c0sf.bwz', 'product_modelname'] = 1
    df.loc[df['product_category']=='55us660h0sd.bwz', 'product_modelname'] = 1
    df.loc[df['product_category']=='43us660h0sd.awz', 'product_modelname'] = 1
    df.loc[df['product_category']=='43us660h (na)', 'product_modelname'] = 1
    df.loc[df['product_category']=='50us660h0sd.bwz', 'product_modelname'] = 1
    df.loc[df['product_category']=='hoteleria_us670h', 'product_modelname'] = 1

    df.loc[df['product_category']=='28mq780', 'product_modelname'] = 1
    df.loc[df['product_category']=='98uh5e', 'product_modelname'] = 1
    df.loc[df['product_category']=='86uh5f', 'product_modelname'] = 1
    df.loc[df['product_category']=='49uh / 49x', 'product_modelname'] = 1
    df.loc[df['product_category']=='49uh / 49xf', 'product_modelname'] = 1

    df.loc[df['product_category']=='55tc3d', 'product_modelname'] = 1
    
    print('product_cagetory를 이용해 product_modelname의 가중치를 수정했습니다.')

In [13]:
# pp_m_c는 product_modelname을 이용하여 product_category의 결측치를 보완하는 함수입니다.
# EDA과정 속에서 product_modelname에는 모델명이 적혀있지만, product_category에는 결측치인 고객이 있다는 것을 발견하였습니다.
# 때문에 이것도 일일이 LG 홈페이지에서 모델명을 일일이 찾아본 후 그에 맞는 category값을 넣어주어 product_category의 결측치를 채웠습니다. 

def pp_m_c(df) :
    df.loc[(df['product_modelname']=='UltraFine Ergo(32UN880)')&df["product_category"].isnull(), 'product_category'] = 'monitor'
    df.loc[(df['product_modelname']=='28MQ780')&df["product_category"].isnull(), 'product_category'] = 'monitor'
    df.loc[(df['product_modelname']=='Ergo Dual(27QP88D)')&df["product_category"].isnull(), 'product_category'] = 'monitor'
    df.loc[(df['product_modelname']=='DualUp(28MQ780)')&df["product_category"].isnull(), 'product_category'] = 'monitor'
    df.loc[(df['product_modelname']=='21HQ513D')&df["product_category"].isnull(), 'product_category'] = 'medical display'
    df.loc[(df['product_modelname']=='UltraWide Ergo(34WN780)')&df["product_category"].isnull(), 'product_category'] = 'monitor'
    df.loc[(df['product_modelname']=='32UN880')&df["product_category"].isnull(), 'product_category'] = 'monitor'
    df.loc[(df['product_modelname']=='31HN713D')&df["product_category"].isnull(), 'product_category'] = 'medical display'
    df.loc[(df['product_modelname']=='55CT5WJ,43HT3WJ,SC-00DA')&df["product_category"].isnull(), 'product_category'] = 'one quick series'
    df.loc[(df['product_modelname']=='31HN713D')&df["product_category"].isnull(), 'product_category'] = 'medical display'
    df.loc[(df['product_modelname']=='32BP95E')&df["product_category"].isnull(), 'product_category'] = 'monitor'
    df.loc[(df['product_modelname']=='55CT5WJ')&df["product_category"].isnull(), 'product_category'] = 'one quick series'
    df.loc[(df['product_modelname']=='43HT3WJ')&df["product_category"].isnull(), 'product_category'] = 'one quick series'
    df.loc[(df['product_modelname']=='LG SuperSign CMS')&df["product_category"].isnull(), 'product_category'] = 'signage care solution'
    df.loc[(df['product_modelname']=='65EP5G OLED Pro')&df["product_category"].isnull(), 'product_category'] = 'oled signage'
    df.loc[(df['product_modelname']=='32BP95E')&df["product_category"].isnull(), 'product_category'] = 'one quick series'
    df.loc[(df['product_modelname']=='55CT5WJ,43HT3WJ')&df["product_category"].isnull(), 'product_category'] = 'one quick series'
    df.loc[(df['product_modelname']=='34WN780')&df["product_category"].isnull(), 'product_category'] = 'monitor'
    df.loc[(df['product_modelname']=='32UN880, 28MQ780')&df["product_category"].isnull(), 'product_category'] = 'monitor'
    df.loc[(df['product_modelname']=='34WN780, 28MQ780')&df["product_category"].isnull(), 'product_category'] = 'monitor'
    df.loc[(df['product_modelname']=='BU60')&df["product_category"].isnull(), 'product_category'] = 'projector'
    df.loc[(df['product_modelname']=='19HK312C')&df["product_category"].isnull(), 'product_category'] = 'medical display'
    df.loc[(df['product_modelname']=='27HJ713C')&df["product_category"].isnull(), 'product_category'] = 'medical display'
    
    df.loc[(df['product_subcategory']=='21HQ513D')&df["product_category"].isnull(), 'product_category'] = 'medical display'
    df.loc[(df['product_subcategory']=='31HN713D')&df["product_category"].isnull(), 'product_category'] = 'medical display'
    df.loc[(df['product_subcategory']=='32BP95E')&df["product_category"].isnull(), 'product_category'] = 'monitor'
    df.loc[(df['product_subcategory']=='27BN85U')&df["product_category"].isnull(), 'product_category'] = 'monitor'
    df.loc[(df['product_subcategory']=='28MQ780')&df["product_category"].isnull(), 'product_category'] = 'monitor'
    df.loc[(df['product_subcategory']=='BU60')&df["product_category"].isnull(), 'product_category'] = 'projector'
    df.loc[(df['product_subcategory']=='19HK312C')&df["product_category"].isnull(), 'product_category'] = 'medical display'
    df.loc[(df['product_subcategory']=='32HL512D')&df["product_category"].isnull(), 'product_category'] = 'medical display'
    df.loc[(df['product_subcategory']=='27HJ713C')&df["product_category"].isnull(), 'product_category'] = 'medical display'  
    
    print('modelname과 subcategory를 이용해 product_category의 결측치를 보완했습니다.')

In [14]:
# pp17은 product_modelname의 전처리 함수입니다.
# 저희는 LG 홈페이지의 Inquiry To Buy 양식을 보면서
# product_category를 선택한 후, product_modelname까지 선택한 고객은 다른 고객에 비해 확실한 니즈가 있는 고객이라고 생각하였습니다.
# 때문에 product_modelname을 적었으면 가중치 1, 안적었으면 0을 주는 전처리를 하였습니다.

def pp17(df) :
    df.loc[df["product_modelname"].isnull(), "product_modelname"] = 0
    df.loc[df["product_modelname"] != 0, "product_modelname"] = 1
    
    if df['product_modelname'].isna().sum() == 0 :
        print('product_modelname의 전처리가 완료되었습니다.')

In [15]:
# pp18은 customer_country.1 칼럼을 전처리 하는 함수입니다.
# 해당 전처리는 약간 복잡하여 번호를 부여하여 설명한 후 해당 부분에 번호 주석을 달아놓겠습니다.

# 우선 1차적인 EDA 결과 특정 행의 EGYPT 값만이 '/'가 없다는 것을 발견했고
# (1) 나중에 한꺼번에 전처리를 편하게 하기위해 EGYPT와 NaN값에 앞에 '//'를 넣어주었습니다.
# (2) 또 이메일많을 적어놓은 고객에 대해서도 '//Others'라는 값을 넣어주었습니다.
# (3) 그리고 '/' 포함 나라의 길이가 4미만인 것들에 대해서는 잘못된 값이라 생각하여 '//Others'로 바꿔주었습니다.
# (4) 그 후에 '/'가 아닌 '.'로 도시와 나라를 구분해 놓은 고객이 있어 이를 '/'로 바꿔주었습니다.

# (5) 이렇게 모든 행이 '/' 2개씩 준비가 된 상태에서 '/'를 기준으로 split 후 맨 뒤의 값만을 가져와 나라값만 가져왔습니다.
# (6) 그 후에 다시 Unique 값들을 확안히여 같은 값이지만 대소문자나 띄워쓰기와 같은 것들로 인해 구별되는 것들을 일일이 확인하며 같게 만들어주었습니다.
# (7) 마지막으로 .value_counts()의 수가 적으며 True에 별로 영향을 안준다고 판단되는 나라들에 대해서 'Others'라는 값을 넣어주었습니다.

def pp18(df) :
    # (1)
    df.loc[df['customer_country.1']=='EGYPT', "customer_country.1"] = '//Egypt'
    df.loc[df["customer_country.1"].isnull(), "customer_country.1"] = "//None"
    
    # (2)
    boolean = []
    for idx in range(len(df.index)):
        if "@" in df.loc[idx, "customer_country.1"].split("/")[-1]:
            boolean.append(True)
        else:
            boolean.append(False) 
    df.loc[boolean, "customer_country.1"] = "//Others"
    
    # (3)
    for i, ctr in enumerate(df['customer_country.1']) :
        if len(ctr) < 4 :
            df['customer_country.1'][i] = "//Others"
    
    # (4)
    for i, ctr in enumerate(df['customer_country.1']) :
        if df['customer_country.1'][i].count('/') != 2 :
            df['customer_country.1'][i] = df['customer_country.1'][i].replace(',', '/')
    
    # (5)
    for i, ctr in enumerate(df['customer_country.1']) :
        df['customer_country.1'][i] = ctr.split('/')[-1]
    
    # (6)
    for i, ctr in enumerate(df['customer_country.1']) :
        if 'India' in ctr :
            df['customer_country.1'][i] = 'India'
        elif ('United States' in ctr) or ('UnitedStates' in ctr) :
            df['customer_country.1'][i] = 'United States'
        elif ('USA' in ctr) or ('U.S.A' in ctr) or ('U.S' in ctr)  or ('US' in ctr)  :
            df['customer_country.1'][i] = 'United States'
        elif 'Brazil' in ctr :
            df['customer_country.1'][i] = 'Brazil'
        elif ('United Kingdom' in ctr) or ('UK' in ctr) or ('U.K' in ctr) :
            df['customer_country.1'][i] = 'United Kingdom'
        elif 'Philippines' in ctr :
            df['customer_country.1'][i] = 'Philippines'
        elif 'Colombia' in ctr :
            df['customer_country.1'][i] = 'Colombia'
        elif 'U.A.E' in ctr :
            df['customer_country.1'][i] = 'U.A.E'
        elif 'Poland' in ctr :
            df['customer_country.1'][i] = 'Poland'
        elif 'Egypt' in ctr :
            df['customer_country.1'][i] = 'Egypt'
        elif 'Chile' in ctr :
            df['customer_country.1'][i] = 'Chile'
        elif 'Italy' in ctr :
            df['customer_country.1'][i] = 'Italy'
        elif ('Saudi Arabia' in ctr) or ('SaudiArabia' in ctr) :
            df['customer_country.1'][i] = 'Saudi Arabia'
        elif 'Canada' in ctr :
            df['customer_country.1'][i] = 'Canada'
        elif 'Argentina' in ctr :
            df['customer_country.1'][i] = 'Argentina'
        elif 'Mexico' in ctr :
            df['customer_country.1'][i] = 'Mexico'
        elif 'Vietnam' in ctr :
            df['customer_country.1'][i] = 'Vietnam'
        elif 'Africa' in ctr :
            df['customer_country.1'][i] = 'South Africa'
        elif 'EE' in ctr :
            df['customer_country.1'][i] = 'United States'
        elif 'New York' in ctr :
            df['customer_country.1'][i] = 'United States'  
        elif 'Singapore' in ctr :
            df['customer_country.1'][i] = 'Singapore' 
        elif 'Germany' in ctr :
            df['customer_country.1'][i] = 'Germany' 
        elif 'Algeria' in ctr :
            df['customer_country.1'][i] = 'Algeria' 
        elif 'Türkiye' in ctr :
            df['customer_country.1'][i] = 'Turkiye' 
        elif 'bangalore' in ctr : 
            df['customer_country.1'][i] = 'Bangalore' 
        elif 'Puerto Rico' in ctr : 
            df['customer_country.1'][i] = 'Puerto Rico' 
        elif 'ALICANTE' in ctr : 
            df['customer_country.1'][i] = 'ALICANTE'
        elif 'Telangana' in ctr :
            df['customer_country.1'][i] = 'Telangana'
        elif 'Panama' in ctr :
            df['customer_country.1'][i] = 'Panama'
        elif 'Indonesia' in ctr :
            df['customer_country.1'][i] = 'Indonesia'
        elif 'Thailand' in ctr :
            df['customer_country.1'][i] = 'Thailand'
            
    for i, ctr in enumerate(df['customer_country.1']) :
        if ctr == '' :
            df['customer_country.1'][i] = 'None'
            
    for idx in range(len(df.index)) :
         if any(char.isdigit() for char in df.loc[idx, "customer_country.1"]):
            df.loc[idx, "customer_country.1"] = "Others"
            
    for i, ctr in enumerate(df['customer_country.1']) :
        if ctr[0] == ' ' :
            df['customer_country.1'][i] = ctr[1:]
            
    df.loc[df['customer_country.1']=='ARGENTINA', "customer_country.1"] = 'Argentina'
    df.loc[df['customer_country.1']=='COLOMBIA', "customer_country.1"] = 'Colombia'
    df.loc[df['customer_country.1']=='PerU', "customer_country.1"] = 'Peru'
    df.loc[df['customer_country.1']=='SPAIN', "customer_country.1"] = 'Spain'
    
    # (7)
    df_pc = df['customer_country.1'].value_counts()
    desired_values = df_pc[df['customer_country.1'].value_counts() < 46]
    idx_list = desired_values.index
    
    for i, con in enumerate(df['customer_country.1']) :
        if con in idx_list :
            df['customer_country.1'][i] = 'Others'
    
    if df['customer_country.1'].isna().sum() == 0 :
        print('customer_country.1의 전처리가 완료되었습니다.(채워진 none값 주의)')

In [16]:
# pp19는 customer_position을 전처리하는 함수입니다.
# 다른 전처리와 마찬가지로 대소문자, 띄워쓰기, 같은 의미를 가지는 단어들에 대해서는 같은 값을 가지게끔 전처리를 해주었습니다.

def pp19(df) :
    
    df.loc[(df['customer_position']=='ceo/founder')|
           (df['customer_position']=='ceo/fundador')|
           (df['customer_position']=='founder')|
           (df['customer_position']=='co-founder')|
           (df['customer_position']=='president')|
           (df['customer_position']=='chairman')|
           (df['customer_position']=='entrepreneurship')|
           (df['customer_position']=='proprietário(a)'), 'customer_position'] = 'CEO/Founder'
    
    df.loc[(df['customer_position']=='c-level executive')|
           (df['customer_position']=='leadership/executive office/owner')|
           (df['customer_position']=='c-levelexecutive')|
           (df['customer_position']=='decision-maker')|
           (df['customer_position']=='decision influencer')|
           (df['customer_position']=='decision-influencer')|
           (df['customer_position']=='decision maker')|
           (df['customer_position']=='chief executive officer'), 'customer_position'] = 'C-level Executive'
    
    df.loc[(df['customer_position']=='manager')|
           (df['customer_position']=='operations')|
           (df['customer_position']=='business development')|
           (df['customer_position']=='av management')|
           (df['customer_position']=='business development/sales')|
           (df['customer_position']=='product management')|
           (df['customer_position']=='genel müdür'), 'customer_position'] = 'Manager'
    
    df.loc[(df['customer_position']=='director')|
           (df['customer_position']=='principal & director')|
           (df['customer_position']=='business unit director')|
           (df['customer_position']=='director cum faculty at gaining apex coaching centre'), 'customer_position'] = 'Director'
    
    df.loc[(df['customer_position']=='associate/analyst')|
           (df['customer_position']=='other - please specify - cedia association'), 'customer_position'] = 'Associate/Analyst'
    
    df.loc[(df['customer_position']=='partner')|
           (df['customer_position']=='business partner'), 'customer_position'] = 'Partner'

    df.loc[(df['customer_position']=='entry level')|
           (df['customer_position']=='entrylevel'), 'customer_position'] = 'Entry Level'
    
    df.loc[(df['customer_position']=='trainee'), 'customer_position'] = 'Trainee'
    
    df.loc[(df['customer_position']=='intern'), 'customer_position'] = 'Intern'
    
    df.loc[(df['customer_position']=='vice president')|
           (df['customer_position']=='vicepresident')|
           (df['customer_position']=='vp'), 'customer_position'] = 'Vice President'
    
    df.loc[(df['customer_position']=='other'), 'customer_position'] = 'others'
    
    df['customer_position'] = df['customer_position'].fillna('none')
        
    if df['customer_position'].isna().sum() == 0 :
        print('Customer Position의 전처리가 완료되었습니다.(채워진 none값 주의)')

In [17]:
# pp21은 expected_timeline을 전처리하는 함수입니다.
# EDA와 LG Inquiry To Buy 확인 결과 expected_timeline을 3개월 단위와 1년 이상으로 나눌 수 있는 것을 확인했으며
# 해당 값으로 나눈 후 나머지 값들에 대해서는 'others'로 두었습니다.

def pp21(df) :
    import numpy as np
    df['expected_timeline'] = df['expected_timeline'].fillna('none')
    df.loc[df["expected_timeline"] == '3_months_~_6_months', "expected_timeline"] = '3 months ~ 6 months'
    df.loc[df["expected_timeline"] == 'less_than_3_months', "expected_timeline"] = 'less than 3 months'
    df.loc[df["expected_timeline"] == 'less than 3 months ,meeting with the customer for the more details and tentative boq will ne 32 and 43', "expected_timeline"] = 'less than 3 months'
    df.loc[df["expected_timeline"] == '3 months', "expected_timeline"] = '3 months ~ 6 months'
    df.loc[df["expected_timeline"] == '9_months_~_1_year', "expected_timeline"] = '9 months ~ 1 year'
    df.loc[df["expected_timeline"] == '6_months_~_9_months', "expected_timeline"] = '6 months ~ 9 months'
    df.loc[df["expected_timeline"] == 'less then 6 months', "expected_timeline"] = '3 months ~ 6 months'
    df.loc[df["expected_timeline"] == 'less than 6 months', "expected_timeline"] = '3 months ~ 6 months'
    df.loc[df["expected_timeline"] == 'more_than_a_year', "expected_timeline"] = 'more than a year'
    df.loc[df["expected_timeline"] == 'more then 3 months', "expected_timeline"] = '3 months ~ 6 months'
    df.loc[df["expected_timeline"] == 'less than 3 months. customer not answered . to call back', "expected_timeline"] = 'less than 3 months'
    df.loc[df["expected_timeline"] == 'one month', "expected_timeline"] = 'less than 3 months'
    df.loc[df["expected_timeline"] == '45 days', "expected_timeline"] = 'less than 3 months'
    df.loc[df["expected_timeline"] == 'less than 5 months', "expected_timeline"] = '3 months ~ 6 months'
    df.loc[df["expected_timeline"] == 'duplicate lead - il220100042906. less than 3 months', "expected_timeline"] = 'less than 3 months'
    df.loc[df["expected_timeline"] == '9 months - 1 year', "expected_timeline"] =  '9 months ~ 1 year'
    df.loc[(df["expected_timeline"] !='less than 3 months')&(df["expected_timeline"] !='3 months ~ 6 months')&(df["expected_timeline"] !='more than a year')&(df["expected_timeline"] !='9 months ~ 1 year')&(df["expected_timeline"] !='6 months ~ 9 months')&(df["expected_timeline"] !='none'), "expected_timeline"] = 'others'
    
    print('expected_timeline의 전처리가 완료되었습니다.(채워진 none값 주의)')

In [18]:
# pp_12_19는 customer_job(pp12)과 customer_position(pp19)의 결측치를 서로 보완해주는 전처리 함수입니다.
# customer_job과 customer_position의 값이 의미하는 바는 느낌이 서로 다르지만 직급에 대해서는 비슷한 의미를 가졌다고 생각했습니다.
# 그래서 서로의 결측치 중에서 키워드 중심으로 채워줄 수 있는 것이 있다면 채워주는 쪽으로 전처리를 진행하였습니다.

def pp_12_19(df) :
    for i, pos in enumerate(df['customer_position']) :
        if (df['customer_job'][i]=='none') :
            if pos == 'CEO/Founder' :
                df['customer_job'][i] = 'ceo'
            elif pos == 'Manager' :
                df['customer_job'][i] = 'manager'
            elif pos == 'Director' :
                df['customer_job'][i] = 'director'
            elif pos == 'C-level Executive' :
                df['customer_job'][i] = 'executive'
            elif ('health' in pos) or ('clinic' in pos) or ('medi' in pos) or ('surger' in pos)or ('hospi' in pos) :
                df['customer_job'][i] = 'medical'
            elif pos == 'consultant' :
                df['customer_job'][i] = 'consulting'
            elif ('teach' in pos) or ('edu' in pos) or ('prof' in pos) :
                df['customer_job'][i] = 'education'
            elif 'tech' in pos :
                df['customer_job'][i] = 'technical'
                
    for i, job in enumerate(df['customer_job']) :
        if (df['customer_position'][i]=='none') :
            if job == 'ceo' :
                df['customer_position'][i] = 'CEO/Founder'
            elif job == 'executive':
                df['customer_position'][i] = 'C-level Executive'
            elif job == 'director':
                df['customer_position'][i] = 'Director'
            elif 'manager' in job:
                df['customer_position'][i] = 'Manager'
                
    df.loc[df['customer_job']=='project manager', 'customer_job'] = 'program and project management'
    
    print('customer_job과 customer_position의 결측치가 서로 조금 보완되었습니다.')

In [19]:
# pp_con_ca는 customer_country를 이용하여 product_category의 결측치를 채우는 전처리 함수입니다.
# 계속되는 EDA 중에 customer_country에 product_category의 특정 키워드 값이 있다는 사실을 발견했고 
# 이 값을 이용하여 product_category의 결측치를 조금 보완하였습니다.

def pp_con_ca(df) :
    df['product_category'] = df['product_category'].fillna('none')
    df['customer_country'] = df['customer_country'].fillna('none')
    for i, ctr in enumerate(df['customer_country']) :
        if ('Commercial D' in ctr) and (df['product_category'][i]=='none') :
            df['product_category'][i] = 'commercial display'
        elif 'Cloud IT' in ctr :
            df['product_category'][i] = 'cloud device'
        elif ('Medical' in ctr) or ('Health' in ctr) or ('Hospital' in ctr) :
            df['business_area'][i] = 'hospital & health care'
    print('country를 이용하여 product_category의 결측치를 조금 보완하였습니다.')

In [20]:
# pp_final은 지금까지 한 각 칼럼별 전처리 함수를 하나로 모으는 함수입니다.
# 기존에 pp17에서 product_modelname만 적은 사람에게 가중치 1을 준 것에 더해 product_subcategory까지 적은 사람에게도 가중치 1을 주는 코드를 추가했으며
# 나머지 결측치를 채워주고 type을 그에 맞게 맞추어주는 코드가 추가되었습니다.

def pp_final(df) :
    df.loc[~(df["product_subcategory"].isnull()), "product_modelname"] = 1
    pp_con_ca(df)
    pp_m_c(df)
    pp_drop(df)
    pp2(df)
    pp6(df)
    pp8(df)
    pp12(df)
    pp13(df)
    pp14(df)
    pp15_hu(df)
    pp15(df)
    pp17(df)
    pp18(df)
    pp19(df)
    pp21(df)
    pp_12_19(df)
    pp6_hu(df)

    df['com_reg_ver_win_rate'] = df['com_reg_ver_win_rate'].fillna(0)
    df['id_strategic_ver'] = df['id_strategic_ver'].fillna(0)
    df['id_strategic_ver'] = df['id_strategic_ver'].astype(int)
    df['it_strategic_ver'] = df['it_strategic_ver'].fillna(0)
    df['it_strategic_ver'] = df['it_strategic_ver'].astype(int)
    df['idit_strategic_ver'] = df['idit_strategic_ver'].fillna(0)
    df['idit_strategic_ver'] = df['idit_strategic_ver'].astype(int)    
    df['ver_win_rate_x'] = df['ver_win_rate_x'].fillna(0)
    df['ver_win_ratio_per_bu'] = df['ver_win_ratio_per_bu'].fillna(0)
    df['business_area'] = df['business_area'].fillna('none')
    print('가중치, 파생변수들의 결측치를 0으로 채웠습니다.')
    
    df.loc[df['customer_job']=='technical', 'customer_job'] = 'others'
    print()
    print('전처리 전부 완료!')
    
    return df

In [21]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('submission.csv')

In [22]:
del df_test['id']
df_all = pd.concat([df_train[df_test.columns], df_test[df_test.columns]], ignore_index=True)

In [23]:
# train data와 test data를 불러온 후 concat을 하고 한꺼번에 전처리를 시작합니다.
df_all_pp = pp_final(df_all)

country를 이용하여 product_category의 결측치를 조금 보완하였습니다.
modelname과 subcategory를 이용해 product_category의 결측치를 보완했습니다.
drop이 완료되었습니다.
customer_type 전처리가 완료되었습니다.(채워진 none값 주의)
historical_existing_cnt 전처리가 완료되었습니다.
customer_job 전처리가 완료되었습니다.(채워진 none값 주의)
lead_desc_length 전처리가 완료되었습니다.
inquiry_type 전처리가 완료되었습니다.(채워진 none값 주의)
product_cagetory를 이용해 product_modelname의 가중치를 수정했습니다.
product_category의 전처리가 완료되었습니다.(채워진 none값 주의)
product_modelname의 전처리가 완료되었습니다.
customer_country.1의 전처리가 완료되었습니다.(채워진 none값 주의)
Customer Position의 전처리가 완료되었습니다.(채워진 none값 주의)
expected_timeline의 전처리가 완료되었습니다.(채워진 none값 주의)
customer_job과 customer_position의 결측치가 서로 조금 보완되었습니다.
customer_type, customer_pos 후처리가 완료되었습니다.
가중치, 파생변수들의 결측치를 0으로 채웠습니다.

전처리 전부 완료!


In [24]:
# 다시 train과 test로 나눠 저장합니다.
df_train_pp = df_all_pp.iloc[: len(df_train)]
df_test_pp = df_all_pp.iloc[len(df_train) :]

In [25]:
df_train_pp.to_csv("df_train_pp_final_nal.csv", index=False)
df_test_pp.to_csv("df_test_pp_final_nal.csv", index=False)

여기까지가 기본적인 저희팀의 전처리 코드입니다. <br>
이제부터 데이터를 활용하여 이진 분류 모델을 만드는 코드입니다. <br>
<br>
약간의 요약을 하자면,<br>
저희 팀은 lead_owner을 활용한 가중치 칼럼을 따로 만들어 추가하였고<br>
min-max 알고리즘과 오버샘플링으로 ADASYN을 적용하였습니다.<br>
모델은 class_weight를 적용한 xgb와 catboost를 사용하였습니다.<br>
<br>
많은 실험 중,<br>
t-SNE나 UMAP과 같은 수학적 기법도 활용하여 모델의 정확도를 높일려고 시도하였으나 오히려 점수가 떨어졌고<br>
오토글로운이나 optuna는 오버피팅이 나는 것 같아 사용하지 않았습니다.<br>

In [26]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [27]:
# leadowner 함수는 추가적인 파생변수를 생성하는 함수입니다.
# EDA 결과 lead_owner에 true 영향을 미치는 요소가 정말 많은 것을 확인하였습니다.
# 때문에 train data 기준 true비율이 0.3이상이면서, value_counts()수가 3이상인 친구들에 대해 가중치를 주었습니다.

def leadowner(df):
    idx = []
    idx_test = []
    
    df["leadowner_weight"] = 0
    
    for name in df["lead_owner"].unique():
        True_sum = sum(df.loc[(df["lead_owner"] == name) & (df["is_converted"] == True), "is_converted"])
        length = sum(df["lead_owner"] == name)
        success_ratio = True_sum / length
        
        cond = (success_ratio > 0.3) & (length > 3)
        
        if cond:
            df.loc[df["lead_owner"] == name, "leadowner_weight"] = 1
            
            if (success_ratio >= 0.8):
                idx.append(name)
                
            idx_test.append(name)
                
    return df, idx, idx_test

In [28]:
# 전처리한 데이터 불러오기
df_train = pd.read_csv("df_train_pp_final_nal.csv") # 학습용 데이터
df_test = pd.read_csv("df_test_pp_final_nal.csv") # 테스트 데이터(제출파일의 데이터)

In [29]:
# leadowner 함수 실행
df_train, ind, ind_test = leadowner(df_train)

In [30]:
# test data에도 lead_owner 가중치 주기
df_test["leadowner_weight"] = 0

for name in ind_test:
    df_test.loc[df_test["lead_owner"] == name, "leadowner_weight"] = 1

In [31]:
# df_train의 is_converted 저장
convert_is = df_train["is_converted"]

In [32]:
df_train.drop("is_converted", axis = 1, inplace = True)
df_test.drop("is_converted", axis = 1, inplace = True)

In [33]:
# label encoder 정의
encoder = LabelEncoder()

label_columns = [
    "business_unit",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "customer_position",
    "response_corporate",
    "lead_desc_length",
    "product_category",
    "expected_timeline",
    "customer_type",
    "customer_country.1",
    "historical_existing_cnt",
    "business_area",
    "bant_submit"
    ]

In [34]:
# concat 후 label encoding
df_all = pd.concat([df_train, df_test], ignore_index = True)

In [35]:
for i in label_columns:
    encoder = LabelEncoder()
    encoder.fit(df_all[i])
    df_all[i] = encoder.transform(df_all[i])

In [36]:
# label encoding 완료 후 train과 test split
df_train1 = df_all.iloc[: len(df_train)]
df_test1 = df_all.iloc[len(df_train) :]

In [37]:
df_train1["is_converted"] = convert_is

In [38]:
# data에 min-max 알고리즘 적용
df_train1 = (df_train1 - df_train1.min())/(df_train1.max() - df_train1.min())
df_test1 = (df_test1 - df_test1.min())/(df_test1.max() - df_test1.min())

In [39]:
# Train data에 True비율이 매우 적어 OverSampling 적용
# 직접 다양한 방법으로 oversampling 하는 방법과 borderline smote등 많이 해보았지만 ADASYN이 성능이 가장 좋았습니다.

smote = ADASYN(n_neighbors = 29, sampling_strategy = {True : 40000}, random_state = 25)

In [40]:
X_resampled, y_resampled = smote.fit_resample(df_train1.drop("is_converted", axis = 1), df_train1["is_converted"])

In [41]:
df_train1 = pd.concat([X_resampled, y_resampled], axis = 1)

In [42]:
# ADASYN을 적용하면 가중치 칼럼에 0과 1이 아닌 값이 생성되므로 이에 해당하는 행들은 drop

drop_idx = ((df_train1["id_strategic_ver"] != 1) & (df_train1["id_strategic_ver"] != 0)) | ((df_train1["ver_cus"] != 1) & (df_train1["ver_cus"] != 0)) | ((df_train1["ver_pro"] != 1) & (df_train1["ver_pro"] != 0)) | ((df_train1["leadowner_weight"] != 1) & (df_train1["leadowner_weight"] != 0)) | ((df_train1["enterprise"] != 1) & (df_train1["enterprise"] != 0))
df_train1 = df_train1.loc[~drop_idx, :].reset_index(drop = True)

In [43]:
df_test1.index = df_test.index

In [44]:
# train, test split
x_train, x_val, y_train, y_val = train_test_split(
    df_train1.drop("is_converted", axis=1),
    df_train1["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state = 25
)

In [45]:
# 저희팀은 xgboost와 catboost만을 사용하여 voting 하였습니다.
# 학습 도중 True에 좀 더 비중을 주기 위해 class_weight를 주었습니다.

models = [
    ('xgb', xgb.XGBClassifier(objective='binary:logistic', class_weights = {0 : 1.0, 1 : 20.0}, random_state = 117)),
    ('cat', cat.CatBoostClassifier(iterations = 1200, learning_rate = 0.05, depth = 8, verbose = False, class_weights = {0 : 1.0, 1 : 20.0}, random_state = 117))
]

In [46]:
model = VotingClassifier(models, voting='soft')
model.fit(x_train, y_train)

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            class_weights={0: 1.0, 1: 20.0},
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction...
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x0000021BCD4ADDF0>)],
                 voting='soft')

In [47]:
pred = model.predict(x_val)
get_clf_eval(y_val, pred)

오차행렬:
 [[ 7253    78]
 [  437 10402]]

정확도: 0.9717
정밀도: 0.9432
재현율: 0.9894
F1: 0.9657


In [48]:
x_test = df_test1.copy()

test_pred = model.predict(x_test)

# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")

df_sub["is_converted"] = test_pred

In [49]:
sum(test_pred)

1912.0

In [50]:
df_sub.to_csv("submission.csv", index=False)